# This notebook gives examples of measuring centroids and doing photometry, both with PhotUtils and Source Extractor.  It is designed as a demo, and will not be collected.

## Part 1: Calculating Centroids

In [ ]:
import numpy as np
from astropy.convolution import AiryDisk2DKernel
import matplotlib.pyplot as plt

Let's define a source as an Airy Disk, and put it in a 101x101 pixel image.

We'll choose a "radius" of 5 pixels (this is the distance from the center to the first negative).

In [ ]:
source=np.array(AiryDisk2DKernel(5,x_size=31,y_size=31))
source=source/np.sum(source)*1000.0  #Normalize the total flux of the source to 1000

image=np.zeros([101,101])
image[50:81,10:41]=source  #y,x, note y starts from top
image=image+1.0 #Lets add a sky background of 2 e-
image=np.random.poisson(image)  #Sample the image with Poisson noise
image=image+np.random.normal(0.0,1.0,[101,101])  #Add a read noise of 1 e-

In [ ]:
#Display the image
#Make sure the source looks like you expected

plt.figure()
plt.imshow(image,vmin=0,vmax=0.5*np.max(image))
plt.colorbar()
plt.tight_layout()

Lets first measure the centroids manually by calculating the marginalized distribution of the PSF on each axis.

This is the first set of equations on slide 4 of the lecture slides.

First, lets collapse in the y-direction to find the x centroid.

(We will sum over the full image since we only have one source, but if you had >1, you would have to narrow down the sum range to stick to the region near your source.)

In [ ]:
x_marg=np.zeros(101)  #Define an empty array to store our marginalized array
for i in range(101):  #Loop over the x pixels, summing all pixels in the y-direction
    x_marg[i]=np.sum(image[:,i])
#Then do the same in the y-direction
y_marg=np.zeros(101)
for j in range(101):
    y_marg[j]=np.sum(image[j,:])
    
#Make a plot to look at these marginalized distributions
plt.figure()
plt.plot(np.arange(101),x_marg,'blue',label='x')
plt.plot(np.arange(101),y_marg,'red',label='y')
plt.legend()
plt.tight_layout()

Ok, I see the peak in x is around 25, and in y is around 65.  
Is this what we would expect from how we defined the image?

### Now, lets try a more accurate calculation, which is the intensity weighted centroid.  
First, we need to calculate the mean intensity in each direction as the total of the marginalized distributions, divided by the number of pixels you're averaging over.  This is the second set of equations on slide 4 of the lecture slides.  
We'll make use of the x_marg and y_marg variables we calculated above, as they are the sums in the equations, which is the same for this calculation.

In [ ]:
n_pix=(len(x_marg)-1.)/2.
x_mean=(1/(2*n_pix+1))*np.sum(x_marg)  
y_mean=(1/(2*n_pix+1))*np.sum(y_marg)  

Finally, calculate the intensity-weighted centroid.

In [ ]:
xarr=np.arange(101) ; yarr=np.arange(101)  #Set up an integer array of x and y values
#Calculate the argument x(y)_marg-x(y)_mean, set minimum to zero
xsum=x_marg-x_mean ; xsum=xsum.clip(min=0)
ysum=y_marg-y_mean ; ysum=ysum.clip(min=0)

#Evaluate the intensity-weighted centroid
xcen=np.sum((xsum)*xarr)/np.sum((xsum))
ycen=np.sum((ysum)*yarr)/np.sum((ysum))

#Print the x and y centers, and compare to your plot above - do they look right?  If not, why not?
print(xcen,ycen)

Measure centroids using three different canned routines, verify they agree with manual. You may need to pip install photutils. You can ignore the deprecation warnings.

In [ ]:
from photutils import centroid_com, centroid_1dg, centroid_2dg

In [ ]:
x, y = centroid_com(image)    #This is the moment distribution, like we did above
x2, y2 = centroid_1dg(image)  #This fits a Gaussian separtely in the x and y directions
x3, y3 = centroid_2dg(image)  #This fits a 2D Gaussian to the full image

#Print all three values; do they agree with your calculated version?
print(x,y)
print(x2,y2)
print(x3,y3)

## Part 2: Aperture Photometry

In [ ]:
#Perform aperture photometry at the position calculated above

from photutils import CircularAperture
from photutils import aperture_photometry

In [ ]:
positions = (x3,y3) #Set the centroid of our source using the most accurate value from above
apertures = CircularAperture(positions, r=5.)  #Use an aperture with r=5 pixels (~FWHM)

phot_table = aperture_photometry(image, apertures) #Do the actual aperture photometry
print(phot_table) 


This prints our results, showing the source number, x and y position, and the photometry measure.

If we wanted just the actual photometric value, this is stored in: phot_table['aperture_sum']

In [ ]:
#Perform photometry in annulus around the position above
from photutils import CircularAnnulus

In [ ]:
#First, define an annulus - r_in and r_out set position of annulus
annulus_apertures = CircularAnnulus(positions, r_in=8., r_out=10.)
#Measure the flux in the annulus
annulus_table = aperture_photometry(image, annulus_apertures)
print(annulus_table)

Efficiency tool - we can actually measure thre flux in our aperture and in the annulus at the same time

In [ ]:

apers=[apertures,annulus_apertures]  #Define a list of apertures; first is science, second is background annulus
final_table = aperture_photometry(image, apers)
print(final_table)


Now, final_table['aperture_sum_0'] is the flux of our source of interest, and
    final_table['aperture_sum_1'] is the *total* flux in our annulus.
    
Finally, to get the flux of our source, we need to calculate the background per pixel, and subtract it.

Background in counts/pix=annulus flux divided by npix in annulus - we can use the attribute "area", i.e. annulus_apertures.area()

In [ ]:
bkg_mean = float(final_table['aperture_sum_1'][0]) / annulus_apertures.area()
bkg_sum = bkg_mean * apertures.area()  #Total background in our science aperture
final_sum = final_table['aperture_sum_0'][0] - bkg_sum  #Do background subtraction
final_table['Final_aperture_flux'] = final_sum  #Add column to table with this final flux
print(final_table)

final_table['Final_aperture_flux'] is our final flux measurement, which can be used for science.

## Part 3: Source Extractor
### In class, we showed how to take a reduced 30" CCD image, add world coordinate system (WCS) to the header via astrometry.net, and how to make a catalog with Source Extractor.  Here, we will read in this catalog, and figure out how to calculate the fluxes of objects in physical units (e.g., erg/s/cm2/Hz), which requires calculating a magnitude zeropoint.

In [ ]:
#Read in catalog - ascii automagically figures out its a Source Extractor Catalog
from astropy.io import ascii
cat=ascii.read('qatar1_r.cat')

In [ ]:
#View the catalog
cat

In [ ]:
#Examine the spatial distribution of my detected sources.  Any obvious problem?
ra=np.array(cat['ALPHA_J2000'])
dec=np.array(cat['DELTA_J2000'])
plt.figure()
plt.scatter(ra,dec,c='red',marker='.',s=0.1)
plt.tight_layout()

Let's check out a magnitude histogram, using FLUX_AUTO, which is the flux in counts measured in the MAG_AUTO elliptical aperture.

Before turning into a magnitude, we need to search for negative flux values, which will cause problems since magnitude is on a log scale (negative pixels are just noise anyway).
    
    
I set them to tiny tiny numbers, which will make a large magnitude, easily flagged later.

In [ ]:
flux=np.array(cat['FLUX_AUTO'])

g=np.where(flux < 0.0) ; flux[g]=1e-30
mag=(-2.5)*np.log10(flux)
plt.figure()
plt.hist(mag,bins=30,range=[-20,0])
plt.tight_layout(); plt.show(); plt.close()
#This histogram shows our object's *instrumental* magnitudes

We need to find a zeropoint for our image.  To do this, lets match to the PAN-STAARS r-band

In [ ]:
#Import some stuff - you may need to pip install astroquery
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.vizier import Vizier
from astropy.utils.console import ProgressBar

In [ ]:
#Define the columns I would like to get back from Vizier
v = Vizier(columns=['RAJ2000', 'DEJ2000','umag', 'gmag', 'rmag','imag'])

I need to do this in a loop, over each object in my catalog.
This would take forever, so lets just do the first 200 objects.

In [ ]:

ps_rmag=np.zeros(len(ra)) # An empty array to fill with the PS (Pan-STAARS) r-band magnitude for my 200 objects
n=200 #or len(ra) for full catalog

for i in range(n):
    pos = SkyCoord(ra[i]*u.deg,dec[i]*u.deg,frame='icrs') #Make a SkyCoord object for index i of my list of 200 stars
    result=v.query_region(pos,width="60s",catalog='panstaars')  #Query Vizier for sources within 60 arcsec of this position
    #At this point, I have a lot of sources, since many will be within 60"
    #I only want the closest one, so I need to measure the angular distance
    dist=np.zeros(len(result[0]['rmag']))  #Set up an empty distance array
    tra=np.array(result[0]['RAJ2000'])  #Make a variable for the PS RA
    tdec=np.array(result[0]['DEJ2000'])  #Make a variable for the PS Dec
    for j in range(len(dist)): #Run a loop over the PS objects to measure the distance
        tobj=SkyCoord(tra[j]*u.deg,tdec[j]*u.deg,frame='icrs')
        dist[j]=tobj.separation(pos).arcsec
    #Now I have the measured distances.  I want the closest one, but only if its close enough I'm sure its the same object
    #I'll be a little liberal, and assume I've found the closest one if its at < 10"
    g=np.where((dist <= 10.0) & (dist==np.min(dist)))
    if len(g[0]) > 0: #If there is a match within 10", then put the PS r-band magnitude in my array
        ps_rmag[i]=float(result[0]['rmag'][g])
    else: #If there is no match, set the mag to some crazy value nowhere near a real value
        ps_rmag[i]=-99
#This will throw some warnings; its ok

In [ ]:
#Now lets compare the PS r-band magnitudes to our instrumental magnitudes
g=np.where(ps_rmag > 0)  #Only consider objects where I found a match
plt.figure()
plt.scatter(ps_rmag[g],ps_rmag[g]-mag[g],c='red',marker='.',s=0.5)
plt.xlim(10,25)
plt.ylim(23,29)
plt.tight_layout()

We see a pretty good correlation!
The median of the y-axis value is then our zeropoint!
To make sure we only use pretty good values to calculate that zeropoint, let's
calculate the median in a box surrounding the nice sequence.

In [ ]:

g2=np.where((ps_rmag > 14) & (ps_rmag < 17.5) & ((ps_rmag-mag) >= 25.5) & ((ps_rmag-mag) <= 26.5))

zp=np.nanmedian(ps_rmag[g2]-mag[g2])
plt.figure()
plt.scatter(ps_rmag[g],ps_rmag[g]-mag[g],c='red',marker='.',s=0.5)
plt.axhline(y=zp, color='b', linestyle='-')
plt.xlim(10,25)
plt.ylim(23,29)
plt.tight_layout()

The blue line shows our zeropoint, pretty good!
Let's print the value

In [ ]:
zp

## We can now obtain fluxes in cgs units for our sources.  We have two equations which define AB magnitude:

$$m_{\rm AB} = (-2.5)*\rm log_{10}(flux [counts]) + ZP$$
##### and
$$ m_{\rm AB} = (-2.5)*\rm log_{10}(flux [cgs]) - 48.6 $$


##### where "m_ab" is the AB magnitude, flux[counts] is the flux measured from the image, ZP is the zeropoint we measured above, and flux[cgs] is the quantity we want - the flux density in units of erg/s/cm2/Hz
##### Solve these two equations to obtain flux[cgs] in units of flux[counts], and you should get something like what is in the cell below

In [ ]:
#Calculate the flux in cgs units in the r-band (I'll call this fnu)
fnu=np.array(cat['FLUX_AUTO'])*10.**(-0.4*48.6 - 0.4*zp)

In [ ]:
#Lets plot a histogram of our AB magnitudes, and see how it looks
m_ab=(-2.5)*np.log10(fnu)-48.6

#This will throw a warning due to log(negative number), but thats ok for this plotting purpose
plt.figure()
plt.hist(m_ab,bins=30,range=[15,22])
plt.xlabel('AB Magnitude') ; plt.ylabel('Number')
plt.tight_layout()